In [1209]:
# This notebook will take the housing data scraped earlier and clean/format it for further investigation.
import pandas as pd
import numpy as np
import csv
from matplotlib import pyplot as plt

pd.options.display.max_columns = None

In [1210]:
priceRange = [
    [1, 250000], [250001, 300000], [300001, 350000], [350001, 400000],
    [400001, 450000], [450001, 500000], [500001, 750000], [750001, 1000000],
    [1000001, 2000000], [2000001, 5000000], [5000001, 50000000]
    ]

In [1211]:
# Create function to take in column of dataframe and return modified column
# Function will take in resoFacts.associationFee and determine if its monthly, quarterly, etc and div value by appropriate amount
# Returned dataframe series will just a number representing monthly HOA fee due.

def hoaFees(series: pd.DataFrame):
    paymentFreq = series.apply(lambda x: str(x).strip("$,1234567890 "))
    payment = series.apply(lambda x: str(x).strip("$,-abcdefghijklmnopqrstuvwxyz ").replace(',', ''))
    payment = payment.apply(lambda x: float(x))
    paymentFreq.replace('', 0, inplace=True)
    for i, freq in enumerate(paymentFreq):
        if freq == 'monthly':
            payment[i] = payment[i]
        elif freq == 'quarterly':
            payment[i] = payment[i]/4
        elif freq == 'semi-annually':
            payment[i] = payment[i]/6
        elif freq == 'annually':
            payment[i] = payment[i]/12
        else:
            payment[i] = 0    
    payment = round(payment)
    return payment


# Concat partial URL with website prefix
def fullURL(hdpURL: str):
    hdpURL = 'www.zillow.com' + str(hdpURL)
    return hdpURL


# Set area units to a consistent value of 'Square Feet'
def areaUnitConversion(units: pd.DataFrame):
    # Input validation
    if 'lotAreaUnits' not in units.columns or 'lotAreaValue' not in units.columns:
        raise ValueError("Input DataFrame must have 'lotAreaUnits' and 'lotAreaValue' columns.")
    # Create a copy of the DataFrame to avoid in-place changes
    units_copy = units.copy(deep=True)
    for idx in units_copy.index:
        if units_copy.loc[idx, 'lotAreaUnits'] == 'Acres':
            units_copy.loc[idx, 'lotAreaValue'] *= 43560
            units_copy.loc[idx, 'lotAreaUnits'] = 'Square Feet'
        elif units_copy.loc[idx, 'lotAreaUnits'] == 'sqft':
            units_copy.loc[idx, 'lotAreaUnits'] = 'Square Feet'
    return units_copy


def timeOnZillowConversion(toz: str):
    units = toz.strip(' 1234567890')
    if units == 'day':
        units = toz
    elif units == 'days':
        units = toz
    else:
        units = '1 day'
    return units


'''
Function to calculate outliers and return list of outliers and non-outliers.
'''
def outliersAll(sortedList: pd.DataFrame):
    outlierBoolean = []
    Q1 = np.percentile(sortedList, 25)
    Q3 = np.percentile(sortedList, 75)
    IQR = Q3 - Q1
    upperBound = Q3 + 1.5*IQR
    lowerBound = Q1 - 1.5*IQR
    for value in sortedList:
        if (value < lowerBound or value > upperBound):
            outlierBoolean.append(1)
        else:
            outlierBoolean.append(0)
    return outlierBoolean


def outliersSpecific(sortedList: pd.DataFrame, column: str):
    outlierBoolean = [0] * sortedList.shape[0]
    for zipcode in sortedList['zipcode'].unique():
        count = 0
        zipcodeIndex = sortedList.index[sortedList['zipcode']==zipcode]
        Q1 = np.percentile(sortedList[column].loc[sortedList['zipcode']==zipcode], 25)
        Q3 = np.percentile(sortedList[column].loc[sortedList['zipcode']==zipcode], 75)
        IQR = Q3 - Q1
        upperBound = Q3 + 1.5*IQR
        lowerBound = Q1 - 1.5*IQR
        for TF in zipcodeIndex:
            if (sortedList[column][TF] < lowerBound or sortedList[column][TF] > upperBound):
                outlierBoolean[TF] = 1
                count+=1
    return outlierBoolean

'''
Function to take any boolean feature and deal with blanks
'''
def cleanBoolean(feature: pd.Series, replaceBlank=False):
    feature = feature.replace(r'\s+', replaceBlank, regex=True)
    feature = feature.replace(np.nan, replaceBlank)
    return feature

def cleanValue(feature: pd.Series, replaceBlank: dict):
    feature = feature.fillna(value=replaceBlank.get('replace_value'))
    feature.replace(r'\s+', replaceBlank.get('replace_value'), regex=True, inplace=True)
    return feature

'''
Parse JSON format for school rankings
'''
def schoolScores(schools: pd.Series):
    finalRating = []
    finalDistance = []
    for i in range(0, schools.shape[0]):
        distance = []
        grades = []
        rating = []
        level = []
        item = eval(schools.iloc[i])
        for j in range(0, len(item)):
            distance.append(item[j].get('distance'))
            grades.append(item[j].get('grades'))
            rating.append(item[j].get('rating'))
            level.append(item[j].get('level'))
        schoolDict = {'distance':distance, 'grades':grades, 'rating':rating, 'level':level}
        df_school = pd.DataFrame(schoolDict)
        df_school = df_school[['distance', 'rating']].groupby(df_school['level']).max().reset_index()
        finalRating.append(df_school['rating'].max())
        finalDistance.append(df_school['distance'].min())
    output = pd.DataFrame({'schoolMaxRating':finalRating, 'schoolMinDistance':finalDistance})
    return output

In [1212]:
for i, price in enumerate(priceRange):
    if i == 0:
        df_forSale = pd.read_csv(f'Raw_Housing_Data/housingData_{price[0]}_{price[1]}.csv',
                                 index_col=0, dtype={'zipcode':str})
        df_recentlySold = pd.read_csv(f'Raw_Housing_Data/housingData_recentlySold_{price[0]}_{price[1]}.csv',
                                      index_col=0, dtype={'zipcode':str})
    else:
        df_forSale = pd.concat([df_forSale, pd.read_csv(f'Raw_Housing_Data/housingData_{price[0]}_{price[1]}.csv',
                                                        index_col=0)], axis=0, ignore_index=True)
        df_recentlySold = pd.concat([df_recentlySold, pd.read_csv(f'Raw_Housing_Data/housingData_recentlySold_{price[0]}_{price[1]}.csv',
                                                                  index_col=0)], axis=0, ignore_index=True)

In [1213]:
columnNames_forSale = list(map(lambda x: x.split(', '), df_forSale.columns))
columnNames_recentlySold = list(map(lambda x: x.split(','), df_recentlySold))

with open('Raw_Housing_Data/dataframeColumnNames_forSale.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(columnNames_forSale)

with open('Raw_Housing_Data/dataframeColumnNames_recentlySold.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(columnNames_recentlySold)

In [1214]:
# Manually select column names I want to keep in my downsized data frame
columnNames_forSale = ['zpid','homeType','homeStatus','timeOnZillow.1','price','streetAddress','zipcode','yearBuilt','bedrooms','bathrooms','livingAreaValue','livingAreaUnits',
                       'lotAreaValue','lotAreaUnits','zestimate','latitude','longitude','favoriteCount','pageViewCount','newConstructionType','resoFacts.associationFee',
                       'listingSubType.isForeclosure','resoFacts.isSeniorCommunity','resoFacts.propertySubType','hdpUrl']

columnNames_recentlySold = ['zpid','homeType','homeStatus','dateSoldString','timeOnZillow.1','price','streetAddress','zipcode','yearBuilt','bathrooms','bedrooms','livingAreaValue','livingAreaUnits',
                            'lotAreaValue','lotAreaUnits','zestimate','latitude','longitude','favoriteCount','pageViewCount','newConstructionType','resoFacts.associationFee',
                            'listingSubType.isForeclosure','resoFacts.isSeniorCommunity', 'resoFacts.propertySubType','hdpUrl']

addtl = ['taxAssessedValue','cityId','rentZestimate','restimateHighPercent','restimateLowPercent','schools','mortgageRates.thirtyYearFixedRate','mortgageRates.fifteenYearFixedRate',
         'mortgageRates.arm5Rate','resoFacts.garageSpaces','resoFacts.hasAdditionalParcels','resoFacts.hasAssociation','resoFacts.hasAttachedGarage',
         'resoFacts.hasAttachedProperty','resoFacts.hasGarage','resoFacts.hasPrivatePool','resoFacts.hasSpa','resoFacts.hasView','resoFacts.hasWaterfrontView','resoFacts.taxAnnualAmount',
         'resoFacts.waterViewYN']

columnNames_forSale = columnNames_forSale + addtl
columnNames_recentlySold = columnNames_recentlySold + addtl

df_forSale_trunc = df_forSale.loc[:,columnNames_forSale]
df_recentlySold_trunc = df_recentlySold.loc[:,columnNames_recentlySold]
del df_forSale, df_recentlySold

In [1215]:
# Do some initial data cleanup
# Set NaN values to 0. Compute quivalent monthly HOA fees to keep data consistent.
df_forSale_trunc['resoFacts.associationFee'] = df_forSale_trunc['resoFacts.associationFee'].fillna(0)
df_recentlySold_trunc['resoFacts.associationFee'] = df_recentlySold_trunc['resoFacts.associationFee'].fillna(0)
df_forSale_trunc['resoFacts.associationFee'] = hoaFees(df_forSale_trunc['resoFacts.associationFee'])
df_recentlySold_trunc['resoFacts.associationFee'] = hoaFees(df_recentlySold_trunc['resoFacts.associationFee'])

# Set NaN values to 'False'. True is already designated by default, where applicable.
df_forSale_trunc['resoFacts.isSeniorCommunity'] = df_forSale_trunc['resoFacts.isSeniorCommunity'].fillna(False)
df_recentlySold_trunc['resoFacts.isSeniorCommunity'] = df_recentlySold_trunc['resoFacts.isSeniorCommunity'].fillna(False)

# Reformat the string representing the property subtype so it can be easily worked on.
# Removing any RV Community data as it is not a valid 'single family home' sub type for this scope.
df_forSale_trunc['resoFacts.propertySubType'] = df_forSale_trunc['resoFacts.propertySubType'].apply(lambda x: str(x).strip("['']"))
df_recentlySold_trunc['resoFacts.propertySubType'] = df_recentlySold_trunc['resoFacts.propertySubType'].apply(lambda x: str(x).strip("['']"))
df_forSale_trunc = df_forSale_trunc.loc[df_forSale_trunc['resoFacts.propertySubType']!='RV Community']
df_recentlySold_trunc = df_recentlySold_trunc.loc[df_recentlySold_trunc['resoFacts.propertySubType']!='RV Community']

# Add prefix to website
df_forSale_trunc['hdpUrl']=df_forSale_trunc['hdpUrl'].apply(fullURL)
df_recentlySold_trunc['hdpUrl']=df_recentlySold_trunc['hdpUrl'].apply(fullURL)

# Convert yearBuilt to integer and drop any rows that have yearBuilt as NaN
df_forSale_trunc.dropna(subset=['yearBuilt'], inplace=True)
df_recentlySold_trunc.dropna(subset=['yearBuilt'], inplace=True)
df_forSale_trunc['yearBuilt']=df_forSale_trunc['yearBuilt'].astype(int)
df_recentlySold_trunc['yearBuilt']=df_recentlySold_trunc['yearBuilt'].astype(int)

# Drop all labels with lotAreaValue = NaN
df_forSale_trunc.dropna(subset=['lotAreaValue'], inplace=True)
df_recentlySold_trunc.dropna(subset=['lotAreaValue'], inplace=True)

# Drop all labels with lat/long = NaN
df_forSale_trunc.dropna(subset=['latitude', 'longitude'], inplace=True)
df_recentlySold_trunc.dropna(subset=['latitude', 'longitude'], inplace=True)

'''
Switch zestimate value from 0 to list price
Shown in EDA section, list price and zestimate are almost 1:1 correlated.
'''
indexList = df_forSale_trunc.index[df_forSale_trunc['zestimate'].isna()].tolist()
df_forSale_trunc.loc[indexList, 'zestimate'] = df_forSale_trunc.loc[indexList, 'price']
indexList = df_recentlySold_trunc.index[df_recentlySold_trunc['zestimate'].isna()].tolist()
df_recentlySold_trunc.loc[indexList, 'zestimate'] = df_recentlySold_trunc.loc[indexList, 'price']
# df_forSale_trunc.fillna(value=value, inplace=True)
# df_recentlySold_trunc.fillna(value=value, inplace=True)
'''End of zestimate update section'''

# Drop all labels with livingAreaValue = NaN -- This is the primary feature for price prediction.
# Could try to do more searches on $/sqft but for now this will suffice.
df_forSale_trunc.dropna(subset=['livingAreaValue'], inplace=True)
df_recentlySold_trunc.dropna(subset=['livingAreaValue'], inplace=True)

# Drop all livingAreaValue rows == 0. Zero here will cause inf. on price/sqft calc. <- breaks feature scaling and model building.
df_forSale_trunc = df_forSale_trunc.loc[df_forSale_trunc['livingAreaValue']>=500]
df_recentlySold_trunc = df_recentlySold_trunc.loc[df_recentlySold_trunc['livingAreaValue']>=500]

# Drop all bathroom and bedroom values of NaN
df_forSale_trunc.dropna(subset=['bathrooms', 'bedrooms'], inplace=True)
df_recentlySold_trunc.dropna(subset=['bathrooms', 'bedrooms'], inplace=True)

# Fill favoriteCount and pageViewCount with 0
value = {'favoriteCount':0, 'pageViewCount':0}
df_forSale_trunc.fillna(value=value, inplace=True)
df_recentlySold_trunc.fillna(value=value, inplace=True)

# Need to set all timeOnZillow.1 values of less than one day to one day.
df_recentlySold_trunc['timeOnZillow.1'].mask(df_recentlySold_trunc['dateSoldString']=='2024-02-15', '1 day', inplace=True)
df_recentlySold_trunc['timeOnZillow.1'] = df_recentlySold_trunc['timeOnZillow.1'].apply(timeOnZillowConversion)
df_forSale_trunc['timeOnZillow.1'] = df_forSale_trunc['timeOnZillow.1'].apply(timeOnZillowConversion)
df_forSale_trunc['timeOnZillow.1'] = df_forSale_trunc['timeOnZillow.1'].apply(lambda x: x.strip(' days')).astype(int)
df_recentlySold_trunc['timeOnZillow.1'] = df_recentlySold_trunc['timeOnZillow.1'].apply(lambda x: x.strip(' days')).astype(int)
df_recentlySold_trunc.rename(columns={'timeOnZillow.1':'daysOnZillow'}, inplace=True)
df_forSale_trunc.rename(columns={'timeOnZillow.1':'daysOnZillow'}, inplace=True)

# Need to set newConstructionType to either True of False
df_recentlySold_trunc['newConstructionType'] = cleanBoolean(df_recentlySold_trunc['newConstructionType'].copy())
df_forSale_trunc['newConstructionType'] = cleanBoolean(df_forSale_trunc['newConstructionType'].copy())

# Need to keep lotAreaUnits and livingAreaUnits consistently set to Square Feet.
# Will need to create a function to deal with these.
df_forSale_trunc[['lotAreaValue','lotAreaUnits']] = areaUnitConversion(df_forSale_trunc[['lotAreaValue','lotAreaUnits']])
df_recentlySold_trunc[['lotAreaValue','lotAreaUnits']] = areaUnitConversion(df_recentlySold_trunc[['lotAreaValue','lotAreaUnits']])

# Convert dateSoldString to datetime object
df_recentlySold_trunc['dateSoldString'] = pd.to_datetime(df_recentlySold_trunc['dateSoldString'])

'''Creating new feature -- price/sqft'''
df_forSale_trunc['price/sqft'] = df_forSale_trunc['price'].astype(float)/df_forSale_trunc['livingAreaValue'].astype(float)
df_recentlySold_trunc['price/sqft'] = df_recentlySold_trunc['price'].astype(float)/df_recentlySold_trunc['livingAreaValue'].astype(float)

df_forSale_trunc.rename(columns={'price':'listPrice'}, inplace=True)

# Set zipcode to integer value
df_forSale_trunc['zipcode'] = df_forSale_trunc['zipcode'].astype(int)
df_recentlySold_trunc['zipcode'] = df_recentlySold_trunc['zipcode'].astype(int)

df_forSale_trunc['resoFacts.hasAdditionalParcels'] = cleanBoolean(df_forSale_trunc['resoFacts.hasAdditionalParcels'].copy())
df_recentlySold_trunc['resoFacts.hasAdditionalParcels'] = cleanBoolean(df_recentlySold_trunc['resoFacts.hasAdditionalParcels'].copy())
df_forSale_trunc['resoFacts.hasAssociation'] = cleanBoolean(df_forSale_trunc['resoFacts.hasAssociation'].copy())
df_recentlySold_trunc['resoFacts.hasAssociation'] = cleanBoolean(df_recentlySold_trunc['resoFacts.hasAssociation'].copy())
df_forSale_trunc['resoFacts.hasAttachedGarage'] = cleanBoolean(df_forSale_trunc['resoFacts.hasAttachedGarage'].copy())
df_recentlySold_trunc['resoFacts.hasAttachedGarage'] = cleanBoolean(df_recentlySold_trunc['resoFacts.hasAttachedGarage'].copy())
df_forSale_trunc['resoFacts.hasAttachedProperty'] = cleanBoolean(df_forSale_trunc['resoFacts.hasAttachedProperty'].copy())
df_recentlySold_trunc['resoFacts.hasAttachedProperty'] = cleanBoolean(df_recentlySold_trunc['resoFacts.hasAttachedProperty'].copy())
df_forSale_trunc['resoFacts.hasGarage'] = cleanBoolean(df_forSale_trunc['resoFacts.hasGarage'].copy())
df_recentlySold_trunc['resoFacts.hasGarage'] = cleanBoolean(df_recentlySold_trunc['resoFacts.hasGarage'].copy())
df_forSale_trunc['resoFacts.hasPrivatePool'] = cleanBoolean(df_forSale_trunc['resoFacts.hasPrivatePool'].copy())
df_recentlySold_trunc['resoFacts.hasPrivatePool'] = cleanBoolean(df_recentlySold_trunc['resoFacts.hasPrivatePool'].copy())
df_forSale_trunc['resoFacts.hasSpa'] = cleanBoolean(df_forSale_trunc['resoFacts.hasSpa'].copy())
df_recentlySold_trunc['resoFacts.hasSpa'] = cleanBoolean(df_recentlySold_trunc['resoFacts.hasSpa'].copy())
df_forSale_trunc['resoFacts.hasView'] = cleanBoolean(df_forSale_trunc['resoFacts.hasView'].copy())
df_recentlySold_trunc['resoFacts.hasView'] = cleanBoolean(df_recentlySold_trunc['resoFacts.hasView'].copy())
df_forSale_trunc['resoFacts.hasWaterfrontView'] = cleanBoolean(df_forSale_trunc['resoFacts.hasWaterfrontView'].copy())
df_recentlySold_trunc['resoFacts.hasWaterfrontView'] = cleanBoolean(df_recentlySold_trunc['resoFacts.hasWaterfrontView'].copy())
df_forSale_trunc['resoFacts.waterViewYN'] = cleanBoolean(df_forSale_trunc['resoFacts.waterViewYN'].copy())
df_recentlySold_trunc['resoFacts.waterViewYN'] = cleanBoolean(df_recentlySold_trunc['resoFacts.waterViewYN'].copy())

value = {'replace_value':0}
df_forSale_trunc['resoFacts.taxAnnualAmount'] = cleanValue(df_forSale_trunc['resoFacts.taxAnnualAmount'].copy(), value)
df_recentlySold_trunc['resoFacts.taxAnnualAmount'] = cleanValue(df_recentlySold_trunc['resoFacts.taxAnnualAmount'].copy(), value)
df_forSale_trunc['resoFacts.garageSpaces'] = cleanValue(df_forSale_trunc['resoFacts.garageSpaces'].copy(), value)
df_recentlySold_trunc['resoFacts.garageSpaces'] = cleanValue(df_recentlySold_trunc['resoFacts.garageSpaces'].copy(), value)

valueMax = {'replace_value':df_forSale_trunc['restimateHighPercent'].max()}
valueMin = {'replace_value':df_forSale_trunc['restimateHighPercent'].min()}
df_forSale_trunc['restimateHighPercent'] = cleanValue(df_forSale_trunc['restimateHighPercent'].copy(), valueMax)
df_forSale_trunc['restimateLowPercent'] = cleanValue(df_forSale_trunc['restimateLowPercent'].copy(), valueMin)

valueMax = {'replace_value':df_recentlySold_trunc['restimateHighPercent'].max()}
valueMin = {'replace_value':df_recentlySold_trunc['restimateHighPercent'].min()}
df_recentlySold_trunc['restimateHighPercent'] = cleanValue(df_recentlySold_trunc['restimateHighPercent'].copy(), valueMax)
df_recentlySold_trunc['restimateLowPercent'] = cleanValue(df_recentlySold_trunc['restimateLowPercent'].copy(), valueMin)

valueRecentlySold = {'replace_value':df_recentlySold_trunc['taxAssessedValue'].median()}
valueForSale = {'replace_value':df_forSale_trunc['taxAssessedValue'].median()}
df_recentlySold_trunc['taxAssessedValue'] = cleanValue(df_recentlySold_trunc['taxAssessedValue'].copy(), valueRecentlySold)
df_forSale_trunc['taxAssessedValue'] = cleanValue(df_forSale_trunc['taxAssessedValue'].copy(), valueForSale)

valueRecentlySold = {'replace_value':list(df_forSale_trunc['cityId'].mode())[-1]}
valueForSale = {'replace_value':list(df_forSale_trunc['cityId'].mode())[-1]}
df_recentlySold_trunc['cityId'] = cleanValue(df_recentlySold_trunc['cityId'].copy(), valueRecentlySold)
df_forSale_trunc['cityId'] = cleanValue(df_forSale_trunc['cityId'].copy(), valueForSale)

valueRecentlySold = {'replace_value':df_recentlySold_trunc['rentZestimate'].mean()}
valueForSale = {'replace_value':df_forSale_trunc['rentZestimate'].mean()}
df_recentlySold_trunc['rentZestimate'] = cleanValue(df_recentlySold_trunc['rentZestimate'].copy(), valueRecentlySold)
df_forSale_trunc['rentZestimate'] = cleanValue(df_forSale_trunc['rentZestimate'].copy(), valueForSale)

valueRecentlySold = {'replace_value':df_recentlySold_trunc['mortgageRates.thirtyYearFixedRate'].mean()}
valueForSale = {'replace_value':df_forSale_trunc['mortgageRates.thirtyYearFixedRate'].mean()}
df_recentlySold_trunc['mortgageRates.thirtyYearFixedRate'] = cleanValue(df_recentlySold_trunc['mortgageRates.thirtyYearFixedRate'].copy(), valueRecentlySold)
df_forSale_trunc['mortgageRates.thirtyYearFixedRate'] = cleanValue(df_forSale_trunc['mortgageRates.thirtyYearFixedRate'].copy(), valueForSale)

valueRecentlySold = {'replace_value':df_recentlySold_trunc['mortgageRates.fifteenYearFixedRate'].mean()}
valueForSale = {'replace_value':df_forSale_trunc['mortgageRates.fifteenYearFixedRate'].mean()}
df_recentlySold_trunc['mortgageRates.fifteenYearFixedRate'] = cleanValue(df_recentlySold_trunc['mortgageRates.fifteenYearFixedRate'].copy(), valueRecentlySold)
df_forSale_trunc['mortgageRates.fifteenYearFixedRate'] = cleanValue(df_forSale_trunc['mortgageRates.fifteenYearFixedRate'].copy(), valueForSale)

valueRecentlySold = {'replace_value':df_recentlySold_trunc['mortgageRates.arm5Rate'].mean()}
valueForSale = {'replace_value':df_forSale_trunc['mortgageRates.arm5Rate'].mean()}
df_recentlySold_trunc['mortgageRates.arm5Rate'] = cleanValue(df_recentlySold_trunc['mortgageRates.arm5Rate'].copy(), valueRecentlySold)
df_forSale_trunc['mortgageRates.arm5Rate'] = cleanValue(df_forSale_trunc['mortgageRates.arm5Rate'].copy(), valueForSale)

In [1216]:
# Reset Index
df_forSale_trunc.reset_index(inplace=True)
df_forSale_trunc.drop(columns=['index'], inplace=True)

df_recentlySold_trunc.reset_index(inplace=True)
df_recentlySold_trunc.drop(columns=['index'], inplace=True)

In [1217]:
df_forSale_trunc = pd.concat([df_forSale_trunc.copy(), schoolScores(df_forSale_trunc['schools'])], axis=1, ignore_index=False)
df_forSale_trunc.drop(columns=['schools'], inplace=True)
df_recentlySold_trunc = pd.concat([df_recentlySold_trunc, schoolScores(df_recentlySold_trunc['schools'])], axis=1, ignore_index=False)
df_recentlySold_trunc.drop(columns=['schools'], inplace=True)

In [1218]:
'''
Check for outliers in list price and sold price.
'''
df_recentlySold_trunc.sort_values(by='price', inplace=True)
soldPrice = df_recentlySold_trunc[['zipcode','price']]
outlierBoolean = outliersSpecific(soldPrice, column='price')
df_recentlySold_trunc.sort_index(inplace=True)
df_recentlySold_trunc.insert(5, "priceOutlier", outlierBoolean, True)

df_forSale_trunc.sort_values(by='listPrice', inplace=True)
salePrice = df_forSale_trunc[['zipcode','listPrice']]
outlierBoolean = outliersSpecific(salePrice, column='listPrice')
df_forSale_trunc.sort_index(inplace=True)
df_forSale_trunc.insert(4, "priceOutlier", outlierBoolean, True)

In [1219]:
df_forSale_trunc.describe(include='all')

,zpid,homeType,homeStatus,daysOnZillow,priceOutlier,listPrice,streetAddress,zipcode,yearBuilt,bedrooms,bathrooms,livingAreaValue,livingAreaUnits,lotAreaValue,lotAreaUnits,zestimate,latitude,longitude,favoriteCount,pageViewCount,newConstructionType,resoFacts.associationFee,listingSubType.isForeclosure,resoFacts.isSeniorCommunity,resoFacts.propertySubType,hdpUrl,taxAssessedValue,cityId,rentZestimate,restimateHighPercent,restimateLowPercent,mortgageRates.thirtyYearFixedRate,mortgageRates.fifteenYearFixedRate,mortgageRates.arm5Rate,resoFacts.garageSpaces,resoFacts.hasAdditionalParcels,resoFacts.hasAssociation,resoFacts.hasAttachedGarage,resoFacts.hasAttachedProperty,resoFacts.hasGarage,resoFacts.hasPrivatePool,resoFacts.hasSpa,resoFacts.hasView,resoFacts.hasWaterfrontView,resoFacts.taxAnnualAmount,resoFacts.waterViewYN,price/sqft,schoolMaxRating,schoolMinDistance
count,2.244000e+03,2244,2244,2244.000000,2244.000000,2.244000e+03,2244,2244.000000,2244.000000,2244.000000,2244.000000,2244.000000,2244,2.244000e+03,2244,2.244000e+03,2244.000000,2244.000000,2244.000000,2244.000000,2244,2244.000000,2244,2244,2244,2244,2.244000e+03,2244.000000,2244.000000,2244.000000,2244.000000,2244.000000,2244.000000,2244.000000,2244.000000,2244,2244,2244,2244,2244,2244,2244,2244,2244,2244.000000,2244,2244.000000,2244.000000,2244.000000
unique,NaN,1,1,NaN,NaN,NaN,2241,NaN,NaN,NaN,NaN,NaN,1,NaN,1,NaN,NaN,NaN,NaN,NaN,2,NaN,2,2,7,2244,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2,2,2,2,2,2,2,2,NaN,2,NaN,NaN,NaN
top,NaN,SINGLE_FAMILY,FOR_SALE,NaN,NaN,NaN,2831 Quentin Ave SE,NaN,NaN,NaN,NaN,NaN,Square Feet,NaN,Square Feet,NaN,NaN,NaN,NaN,NaN,False,NaN,False,False,Single Family Residence,www.zillow.com/homedetails/5650-Stamford-St-Mi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False,False,False,True,False,False,False,False,NaN,False,NaN,NaN,NaN
freq,NaN,2244,2244,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,2244,NaN,2244,NaN,NaN,NaN,NaN,NaN,1868,NaN,2229,2184,2233,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2071,1445,2132,2238,1974,1738,2016,1334,1890,NaN,1793,NaN,NaN,NaN
mean,2.382643e+08,NaN,NaN,70.474599,0.069964,5.677217e+05,NaN,32909.590909,1995.668895,3.517380,2.438503,2010.419340,NaN,7.961050e+05,NaN,5.478106e+05,28.173084,-80.686519,47.241533,865.347148,NaN,34.016488,NaN,NaN,NaN,NaN,3.174238e+05,42408.464349,3053.454119,29.496435,11.360071,6.510190,5.883190,6.884716,3.080660,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3412.310606,NaN,268.002886,5.996881,1.963993
std,5.212294e+08,NaN,NaN,83.608752,0.255144,5.395089e+05,NaN,48.620087,23.920054,0.785604,0.880895,806.566324,NaN,1.613190e+07,NaN,5.081361e+05,0.222387,0.081578,55.101532,891.230078,NaN,84.464093,NaN,NaN,NaN,NaN,2.918053e+05,61033.056616,2281.311847,30.908022,6.079026,0.177605,0.312042,0.260713,25.713816,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10760.882324,NaN,140.245006,2.040056,1.685194
min,4.336783e+07,NaN,NaN,1.000000,0.000000,7.500000e+04,NaN,32754.000000,1884.000000,0.000000,1.000000,540.000000,NaN,0.000000e+00,NaN,7.160000e+04,27.833658,-80.930824,0.000000,8.000000,NaN,0.000000,NaN,NaN,NaN,NaN,1.980000e+03,7433.000000,1200.000000,5.000000,5.000000,5.838000,5.438000,5.792000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,63.004032,2.000000,0.000000
25%,4.345366e+07,NaN,NaN,16.000000,0.000000,3.349990e+05,NaN,32907.000000,1979.000000,3.000000,2.000000,1536.000000,NaN,7.840800e+03,NaN,3.255000e+05,27.989336,-80.729460,14.000000,286.500000,NaN,0.000000,NaN,NaN,NaN,NaN,1.717400e+05,14563.000000,2040.500000,10.000000,6.000000,6.510190,5.855000,6.884716,2.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,814.000000,NaN,198.367009,4.000000,0.900000
50%,5.461233e+07,NaN,NaN,41.000000,0.000000,3.999000e+05,NaN,32909.000000,1999.000000,3.000000,2.000000,1865.000000,NaN,1.001880e+04,NaN,3.900000e+05,28.121854,-80.679285,32.000000,653.000000,NaN,0.000000,NaN,NaN,NaN,NaN,2.702500e+05,32805.000000,2454.000000,19.000000,10.000000,6.540000,5.855000,6.978000,2.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2474.500000,NaN,228.089349,6.000

In [1220]:
df_recentlySold_trunc.describe(include='all')

,zpid,homeType,homeStatus,dateSoldString,daysOnZillow,priceOutlier,price,streetAddress,zipcode,yearBuilt,bathrooms,bedrooms,livingAreaValue,livingAreaUnits,lotAreaValue,lotAreaUnits,zestimate,latitude,longitude,favoriteCount,pageViewCount,newConstructionType,resoFacts.associationFee,listingSubType.isForeclosure,resoFacts.isSeniorCommunity,resoFacts.propertySubType,hdpUrl,taxAssessedValue,cityId,rentZestimate,restimateHighPercent,restimateLowPercent,mortgageRates.thirtyYearFixedRate,mortgageRates.fifteenYearFixedRate,mortgageRates.arm5Rate,resoFacts.garageSpaces,resoFacts.hasAdditionalParcels,resoFacts.hasAssociation,resoFacts.hasAttachedGarage,resoFacts.hasAttachedProperty,resoFacts.hasGarage,resoFacts.hasPrivatePool,resoFacts.hasSpa,resoFacts.hasView,resoFacts.hasWaterfrontView,resoFacts.taxAnnualAmount,resoFacts.waterViewYN,price/sqft,schoolMaxRating,schoolMinDistance
count,5.676000e+03,5676,5676,5676,5676.000000,5676.000000,5.676000e+03,5676,5676.000000,5676.000000,5676.000000,5676.000000,5676.000000,5676,5.676000e+03,5676,5.676000e+03,5676.000000,5676.000000,5676.000000,5676.000000,5676,5676.000000,5676,5676,5676,5676,5.676000e+03,5676.000000,5676.000000,5676.000000,5676.000000,5676.000000,5676.000000,5676.000000,5676.000000,5676,5676,5676,5676,5676,5676,5676,5676,5676,5676.000000,5676,5676.000000,5676.000000,5676.000000
unique,NaN,1,2,NaN,NaN,NaN,NaN,5661,NaN,NaN,NaN,NaN,NaN,1,NaN,1,NaN,NaN,NaN,NaN,NaN,2,NaN,2,2,7,5676,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2,2,2,2,2,2,2,2,NaN,2,NaN,NaN,NaN
top,NaN,SINGLE_FAMILY,RECENTLY_SOLD,NaN,NaN,NaN,NaN,657 Amy Ct SW,NaN,NaN,NaN,NaN,NaN,Square Feet,NaN,Square Feet,NaN,NaN,NaN,NaN,NaN,False,NaN,False,False,Single Family Residence,www.zillow.com/homedetails/1475-Leeward-Ave-SE...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False,False,False,True,False,False,False,False,NaN,False,NaN,NaN,NaN
freq,NaN,5676,5661,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,5676,NaN,5676,NaN,NaN,NaN,NaN,NaN,5671,NaN,5650,5488,5157,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4864,3285,5484,5584,4718,4505,5002,3296,4606,NaN,4414,NaN,NaN,NaN
mean,2.279959e+08,NaN,NaN,2023-10-06 10:40:35.517970432,128.312368,0.060782,4.903831e+05,NaN,32913.250000,1995.156096,2.368851,3.457012,1977.306730,NaN,3.328761e+05,NaN,5.008836e+05,28.187050,-80.689202,2.291755,58.466702,NaN,40.000352,NaN,NaN,NaN,NaN,3.214799e+05,39287.533474,2979.819273,17.629845,12.612579,6.602521,5.991960,6.735064,3.414024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2886.273608,NaN,240.441933,6.088266,1.976286
min,4.336795e+07,NaN,NaN,2023-02-15 00:00:00,1.000000,0.000000,2.200000e+02,NaN,32754.000000,1901.000000,0.000000,0.000000,504.000000,NaN,0.000000e+00,NaN,1.400000e+04,27.833752,-80.928590,0.000000,0.000000,NaN,0.000000,NaN,NaN,NaN,NaN,2.500000e+03,7433.000000,1150.000000,5.000000,5.000000,5.990000,5.779000,5.783000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,0.158730,2.000000,0.100000
25%,4.344609e+07,NaN,NaN,2023-08-18 00:00:00,62.000000,0.000000,3.030000e+05,NaN,32907.000000,1979.000000,2.000000,3.000000,1492.000000,NaN,7.840000e+03,NaN,3.111000e+05,28.012474,-80.732945,0.000000,7.000000,NaN,0.000000,NaN,NaN,NaN,NaN,1.789825e+05,13560.000000,2025.000000,9.000000,8.000000,6.509000,5.958000,6.731000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,821.000000,NaN,186.944930,4.000000,0.900000
50%,4.818874e+07,NaN,NaN,2023-10-18 00:00:00,117.000000,0.000000,3.994275e+05,NaN,32926.000000,1997.000000,2.000000,3.000000,1828.000000,NaN,1.001880e+04,NaN,4.031000e+05,28.162102,-80.685910,0.000000,14.000000,NaN,0.000000,NaN,NaN,NaN,NaN,2.782250e+05,19307.000000,2394.500000,15.000000,11.000000,6.683000,5.969000,6.830000,2.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2305.000000,NaN,218.657384,6.000000,1.500000
75%,1.041316e+08,NaN,NaN,2023-12-08 00:00:00,177.000000,0.000000,5.250000e+05,NaN,32940.000000,2020.000000,3.000000,4.000000,2256.000000,NaN,1.219680e+04,NaN,5.318750e+05,28.342488,-80.639760,0.000000,28.000000,NaN,38.250000,NaN,NaN,NaN,NaN,3.848350e+05,53787.000000

In [1221]:
df_forSale_trunc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2244 entries, 0 to 2243
Data columns (total 49 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   zpid                                2244 non-null   int64  
 1   homeType                            2244 non-null   object 
 2   homeStatus                          2244 non-null   object 
 3   daysOnZillow                        2244 non-null   int64  
 4   priceOutlier                        2244 non-null   int64  
 5   listPrice                           2244 non-null   float64
 6   streetAddress                       2244 non-null   object 
 7   zipcode                             2244 non-null   int64  
 8   yearBuilt                           2244 non-null   int64  
 9   bedrooms                            2244 non-null   float64
 10  bathrooms                           2244 non-null   float64
 11  livingAreaValue                     2244 non-nul

In [1222]:
df_recentlySold_trunc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5676 entries, 0 to 5675
Data columns (total 50 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   zpid                                5676 non-null   int64         
 1   homeType                            5676 non-null   object        
 2   homeStatus                          5676 non-null   object        
 3   dateSoldString                      5676 non-null   datetime64[ns]
 4   daysOnZillow                        5676 non-null   int64         
 5   priceOutlier                        5676 non-null   int64         
 6   price                               5676 non-null   float64       
 7   streetAddress                       5676 non-null   object        
 8   zipcode                             5676 non-null   int64         
 9   yearBuilt                           5676 non-null   int64         
 10  bathrooms                    

In [1223]:
for column_ in df_forSale_trunc.columns:
    nullValueCount = df_forSale_trunc[column_].isnull().sum()
    print(f'Number of NaN values in {column_}: {nullValueCount}')

Number of NaN values in zpid: 0
Number of NaN values in homeType: 0
Number of NaN values in homeStatus: 0
Number of NaN values in daysOnZillow: 0
Number of NaN values in priceOutlier: 0
Number of NaN values in listPrice: 0
Number of NaN values in streetAddress: 0
Number of NaN values in zipcode: 0
Number of NaN values in yearBuilt: 0
Number of NaN values in bedrooms: 0
Number of NaN values in bathrooms: 0
Number of NaN values in livingAreaValue: 0
Number of NaN values in livingAreaUnits: 0
Number of NaN values in lotAreaValue: 0
Number of NaN values in lotAreaUnits: 0
Number of NaN values in zestimate: 0
Number of NaN values in latitude: 0
Number of NaN values in longitude: 0
Number of NaN values in favoriteCount: 0
Number of NaN values in pageViewCount: 0
Number of NaN values in newConstructionType: 0
Number of NaN values in resoFacts.associationFee: 0
Number of NaN values in listingSubType.isForeclosure: 0
Number of NaN values in resoFacts.isSeniorCommunity: 0
Number of NaN values in

In [1224]:
for column_ in df_recentlySold_trunc.columns:
    nullValueCount = df_recentlySold_trunc[column_].isnull().sum()
    print(f'Number of NaN values in {column_}: {nullValueCount}')

Number of NaN values in zpid: 0
Number of NaN values in homeType: 0
Number of NaN values in homeStatus: 0
Number of NaN values in dateSoldString: 0
Number of NaN values in daysOnZillow: 0
Number of NaN values in priceOutlier: 0
Number of NaN values in price: 0
Number of NaN values in streetAddress: 0
Number of NaN values in zipcode: 0
Number of NaN values in yearBuilt: 0
Number of NaN values in bathrooms: 0
Number of NaN values in bedrooms: 0
Number of NaN values in livingAreaValue: 0
Number of NaN values in livingAreaUnits: 0
Number of NaN values in lotAreaValue: 0
Number of NaN values in lotAreaUnits: 0
Number of NaN values in zestimate: 0
Number of NaN values in latitude: 0
Number of NaN values in longitude: 0
Number of NaN values in favoriteCount: 0
Number of NaN values in pageViewCount: 0
Number of NaN values in newConstructionType: 0
Number of NaN values in resoFacts.associationFee: 0
Number of NaN values in listingSubType.isForeclosure: 0
Number of NaN values in resoFacts.isSeni

In [1225]:
# Check for duplicates
print(f'There are {df_forSale_trunc.listPrice.loc[df_forSale_trunc.duplicated()].count()} duplicates in df_forSale_trunc.')
print(f'There are {df_recentlySold_trunc.price.loc[df_recentlySold_trunc.duplicated()].count()} duplicates in df_recentlySold_trunc.')

There are 0 duplicates in df_forSale_trunc.
There are 0 duplicates in df_recentlySold_trunc.


In [1226]:
# Export dataframes to csv
df_forSale_trunc.to_csv('cleaned_forSale_data.csv', sep=',', index=True, encoding='utf-8')
df_recentlySold_trunc.to_csv('cleaned_recentlySold_data.csv', sep=',', index=True, encoding='utf-8')

In [1227]:
del df_forSale_trunc, df_recentlySold_trunc